In [ ]:
import glob
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from sbi import analysis as analysis
import scipy.stats
import seaborn as sns
sns.set_context("talk")

In [ ]:
from signature.utils import compute_metrics
import importlib

In [ ]:
from matplotlib.lines import Line2D

# OU

In [ ]:
ou_true = np.loadtxt("./OU_true_samples_05_1.txt")

In [ ]:
analysis.pairplot(ou_true, limits=[[0.,1.], [-2,2]], figsize=(5, 5))

### Compute metrics

In [ ]:
ns = [i*100 for i in range(1, 11)]
seeds = range(20)

In [ ]:
ou_sig_wds, _, ou_sig_mds = compute_metrics.metrics(ns, seeds,
                                            "./OU/{0}/signature_{1}_1_100_samples.txt",
                                            ou_true, sliced=False)

In [ ]:
ou_gru_wds, _, ou_gru_mds = compute_metrics.metrics(ns, seeds,
                                            "./OU/{0}/gru-resnet_{1}_samples.txt",
                                            ou_true, sliced=False)

In [ ]:
ou_hc_wds, _, ou_hc_mds = compute_metrics.metrics(ns, seeds,
                                            "./OU/{0}/hc_{1}_samples.txt",
                                            ou_true, sliced=False)

In [ ]:
ou_k2_wds, _, ou_k2_mds = compute_metrics.metrics(ns, seeds,
                                            "./OU/{0}/k2_{1}_samples.txt",
                                            ou_true, sliced=False)

### Distances between means

In [ ]:
df = pd.DataFrame.from_dict(ou_sig_mds)
gru_df = pd.DataFrame.from_dict(ou_gru_mds)
hc_df = pd.DataFrame.from_dict(ou_hc_mds)
k2_df = pd.DataFrame.from_dict(ou_k2_mds)

_df = pd.melt(df)
_df["Method"] = "SignatuRE"

_gru_df = pd.melt(gru_df)
_gru_df["Method"] = "GRU-ResNet"
_df = _df.append(_gru_df)

_k2_df = pd.melt(k2_df)
_k2_df["Method"] = "K2-RE"
_df = _df.append(_k2_df)

ax = sns.lineplot(x="variable", y="value", hue="Method", data=_df, ci=95, style="Method",
                  markers=True, dashes=False)

_hc_df = pd.melt(hc_df)
_hc_df["Method"] = "Bespoke ResNet"

ax = sns.lineplot(x="variable", y="value", color="black", data=_hc_df, ci=95, linestyle=":",
                  markers=True, dashes=False)

ax.set_ylabel(r"$\vert{\hat{\theta} - \hat{\theta}_{MH}\vert}^2$")
ax.set_xlabel("Number of simulations")
ax.set_ylim([3e-3,3])
ax.set_yscale('log')

lines = [Line2D([0], [0], marker='o', ls='-', c='steelblue'),
         Line2D([0], [0], marker='X', ls='-', c='darkorange'),
         Line2D([0], [0], marker='', ls=':', c='black'),
         Line2D([0], [0], marker='s', ls='-', c='forestgreen')]

plt.legend(lines, ["SignatuRE", "GRU-ResNet", "Bespoke ResNet", "K2-RE"], loc='upper right', fontsize=12, ncol=2)

### Plot posteriors

In [ ]:
s = 8
fig, axes = plt.subplots(2, 2, figsize=(s, s))

ou_true_df = pd.DataFrame(ou_true)
ou_true_df["Method"] = "MH"
ou_true_df.columns = [r"$\theta_1$", r"$\theta_2$", "Method"]

palette = {"SignatuRE":"blue", "GRU-ResNet":"orange", "K2-RE":"red", "Bespoke ResNet":"green", "MH":"black"}
linestyles = "--"

seeds = range(20)

#############
# SignatuRE #
#############

sigoudf = pd.DataFrame()
for seed in seeds:
    sig = np.loadtxt("./OU/{0}/signature_500_1_100_samples.txt".format(seed))
    sig_df = pd.DataFrame(sig)
    sig_df["Method"] = "SignatuRE"
    sigoudf = sigoudf.append(sig_df)

sigoudf.columns = [r"$\theta_1$", r"$\theta_2$", "Method"]
sigoudf = sigoudf.append(ou_true_df)
sns.kdeplot(data=sigoudf, x=r"$\theta_1$", y=r"$\theta_2$", hue="Method", ax=axes[0,0], palette=palette,
            legend=False, linestyles=linestyles)

##############
# GRU-ResNet #
##############

gruoudf = pd.DataFrame()
for seed in seeds:
    gru = np.loadtxt("./OU/{0}/gru-resnet_500_samples.txt".format(seed))
    gru_df = pd.DataFrame(gru)
    gru_df["Method"] = "GRU-ResNet"
    gruoudf = gruoudf.append(gru_df)

gruoudf.columns = [r"$\theta_1$", r"$\theta_2$", "Method"]
gruoudf = gruoudf.append(ou_true_df)
sns.kdeplot(data=gruoudf, x=r"$\theta_1$", y=r"$\theta_2$", hue="Method", ax=axes[0,1], palette=palette,
            legend=False, linestyles=linestyles)

#############
# Bespoke ResNet #
#############

hcoudf = pd.DataFrame()
for seed in seeds:
    hc = np.loadtxt("./OU/{0}/hc_500_samples.txt".format(seed))
    hc_df = pd.DataFrame(hc)
    hc_df["Method"] = "Bespoke ResNet"
    hcoudf = hcoudf.append(hc_df)

hcoudf.columns = [r"$\theta_1$", r"$\theta_2$", "Method"]
hcoudf = hcoudf.append(ou_true_df)
sns.kdeplot(data=hcoudf, x=r"$\theta_1$", y=r"$\theta_2$", hue="Method", ax=axes[1,0], palette=palette,
            legend=False, linestyles=linestyles)

#########
# K2-RE #
#########

k2oudf = pd.DataFrame()
for seed in seeds:
    k2 = np.loadtxt("./OU/{0}/k2_500_samples.txt".format(seed))
    k2_df = pd.DataFrame(k2)
    k2_df["Method"] = "K2-RE"
    k2oudf = k2oudf.append(k2_df)

k2oudf.columns = [r"$\theta_1$", r"$\theta_2$", "Method"]
k2oudf = k2oudf.append(ou_true_df)

sns.kdeplot(data=k2oudf, x=r"$\theta_1$", y=r"$\theta_2$", hue="Method", ax=axes[1,1], palette=palette,
            legend=False, linestyles=linestyles)

for ax in axes.reshape(-1):
    ax.set_xlim([-0.07, 1.07])
    ax.set_ylim([-2.07, 2.07])

axes[0,0].set_xlabel("")
axes[0,0].set_xticklabels("")

axes[0,1].set_xlabel("")
axes[0,1].set_xticklabels("")

axes[0,1].set_ylabel("")
axes[0,1].set_yticklabels("")

axes[1,1].set_ylabel("")
axes[1,1].set_yticklabels("")
    
space = 0.15
plt.subplots_adjust(wspace=space, hspace=space)

# handles, labels = axes[1,0].get_legend_handles_labels()
labels=["SignatuRE", "GRU-ResNet", "Bespoke ResNet", "K2-RE", "MH"]
fig.legend(lines, labels, fontsize=14, loc=(0.172,0.9), ncol=3)

### Plot WDs

In [ ]:
df = pd.DataFrame.from_dict(ou_sig_wds)
gru_df = pd.DataFrame.from_dict(ou_gru_wds)
hc_df = pd.DataFrame.from_dict(ou_hc_wds)
k2_df = pd.DataFrame.from_dict(ou_k2_wds)

_df = pd.melt(df)
_df["Method"] = "SignatuRE"

_gru_df = pd.melt(gru_df)
_gru_df["Method"] = "GRU-ResNet"
_df = _df.append(_gru_df)

# _hc_df = pd.melt(hc_df)
# _hc_df["Method"] = "Bespoke ResNet"
# _df = _df.append(_hc_df)

_k2_df = pd.melt(k2_df)
_k2_df["Method"] = "K2-RE"
_df = _df.append(_k2_df)

ax = sns.lineplot(x="variable", y="value", hue="Method", data=_df, ci=95,
                  style="Method", markers=True, dashes=False)

_hc_df = pd.melt(hc_df)
_hc_df["Method"] = "Bespoke ResNet"

ax = sns.lineplot(x="variable", y="value", data=_hc_df, ci=95,
                  linestyle=":", color='black', markers=True, dashes=False)

ax.set_ylabel("Wasserstein distance")
ax.set_xlabel("Number of simulations")
ax.set_ylim([0.07,1.27])

lines = [Line2D([0], [0], marker='o', ls='-', c='steelblue'),
         Line2D([0], [0], marker='X', ls='-', c='darkorange'),
         Line2D([0], [0], marker='', ls=':', c='black'),
         Line2D([0], [0], marker='s', ls='-', c='forestgreen')]

plt.legend(lines, ["SignatuRE", "GRU-ResNet", "Bespoke ResNet", "K2-RE"], loc='upper right', fontsize=12, ncol=2)

In [ ]:
df = pd.DataFrame.from_dict(ou_gru6_wds)
gru_df = pd.DataFrame.from_dict(ou_gru_wds)
k2_df = pd.DataFrame.from_dict(ou_gru9_wds)

_df = pd.melt(df)
_df["Method"] = "GRU6"

_gru_df = pd.melt(gru_df)
_gru_df["Method"] = "GRU-ResNet"
_df = _df.append(_gru_df)

# _hc_df = pd.melt(hc_df)
# _hc_df["Method"] = "Bespoke ResNet"
# _df = _df.append(_hc_df)

_k2_df = pd.melt(k2_df)
_k2_df["Method"] = "GRU9"
_df = _df.append(_k2_df)

ax = sns.lineplot(x="variable", y="value", hue="Method", data=_df, ci=95,
                  style="Method", markers=True, dashes=False)

ax.set_ylabel("Wasserstein distance")
ax.set_xlabel("Number of simulations")
ax.set_ylim([0.07,1.27])

lines = [Line2D([0], [0], marker='o', ls='-', c='steelblue'),
         Line2D([0], [0], marker='X', ls='-', c='darkorange'),
         Line2D([0], [0], marker='s', ls='-', c='forestgreen')]

plt.legend(lines, ["GRU6", "GRU", "GRU9"], loc='upper right', fontsize=12, ncol=2)

# MA2

In [ ]:
ma2_true = np.loadtxt("./MA2_samples.txt")

In [ ]:
_, ax = plt.subplots(1,1,figsize=(4,4))
ax.scatter(ma2_true[:,0], ma2_true[:,1], alpha=0.05)
ax.set_xlim([-2.05, 2.05])
ax.set_ylim([-1.05, 1.05])

## Compute metrics

In [ ]:
ma2_sig_wds, _, ma2_sig_mds = compute_metrics.metrics(ns, seeds,
                            "./MA2/{0}/signature_{1}_1_samples.txt",
                                           ma2_true, sliced=False)

In [ ]:
ma2_gru_wds, _, ma2_gru_mds = compute_metrics.metrics(ns, seeds,
                                            "./MA2/{0}/gru-resnet_{1}_samples.txt",
                                            ma2_true, thin=100, sliced=False)

In [ ]:
ma2_hc_wds, _, ma2_hc_mds = compute_metrics.metrics(ns, seeds,
                                            "./MA2/{0}/hc_{1}_samples.txt",
                                            ma2_true, sliced=False)

In [ ]:
ma2_k2_wds, _, ma2_k2_mds = compute_metrics.metrics(ns, seeds,
                                            "./MA2/{0}/k2_{1}_samples.txt",
                                            ma2_true, sliced=False)

### Plot WDs

In [ ]:
df = pd.DataFrame.from_dict(ma2_sig_wds)
gru_df = pd.DataFrame.from_dict(ma2_gru_wds)
hc_df = pd.DataFrame.from_dict(ma2_hc_wds)
k2_df = pd.DataFrame.from_dict(ma2_k2_wds)

_df = pd.melt(df)
_df["Method"] = "SignatuRE"

_gru_df = pd.melt(gru_df)
_gru_df["Method"] = "GRU-ResNet"
_df = _df.append(_gru_df)

_hc_df = pd.melt(hc_df)
_hc_df["Method"] = "Bespoke ResNet"

_k2_df = pd.melt(k2_df)
_k2_df["Method"] = "K2-RE"
_df = _df.append(_k2_df)

ax = sns.lineplot(x="variable", y="value", hue="Method", data=_df, ci=95,
                  style="Method", markers=True, dashes=False)

ax = sns.lineplot(x="variable", y="value", color="black", data=_hc_df, ci=95, linestyle=":",
                  markers=False, dashes=False)

ax.set_ylabel("Wasserstein distance")
ax.set_xlabel("Number of simulations")
ax.set_ylim([0.29,0.98])

lines = [Line2D([0], [0], marker='o', ls='-', c='steelblue'),
         Line2D([0], [0], marker='X', ls='-', c='darkorange'),
         Line2D([0], [0], marker='', ls=':', c='black'),
         Line2D([0], [0], marker='s', ls='-', c='forestgreen')]

plt.legend(lines, ["SignatuRE", "GRU-ResNet", "Bespoke ResNet", "K2-RE"], loc='upper right', fontsize=12, ncol=2)

### Distances between means

In [ ]:
df = pd.DataFrame.from_dict(ma2_sig_mds)
gru_df = pd.DataFrame.from_dict(ma2_gru_mds)
hc_df = pd.DataFrame.from_dict(ma2_hc_mds)
k2_df = pd.DataFrame.from_dict(ma2_k2_mds)

_df = pd.melt(df)
_df["Method"] = "SignatuRE"

_gru_df = pd.melt(gru_df)
_gru_df["Method"] = "GRU-ResNet"
_df = _df.append(_gru_df)

_hc_df = pd.melt(hc_df)
_hc_df["Method"] = "Bespoke ResNet"

_k2_df = pd.melt(k2_df)
_k2_df["Method"] = "K2-RE"
_df = _df.append(_k2_df)

ax = sns.lineplot(x="variable", y="value", hue="Method", data=_df, ci=95, style="Method",
                  markers=True, dashes=False)

ax = sns.lineplot(x="variable", y="value", color="black", data=_hc_df, ci=95, linestyle=":",
                  markers=False, dashes=False)

ax.set_ylabel(r"$\vert{\hat{\theta} - \hat{\theta}_{MH}\vert}^2$")
ax.set_xlabel("Number of simulations")
ax.set_yscale('log')
lines = [Line2D([0], [0], marker='o', ls='-', c='steelblue'),
         Line2D([0], [0], marker='X', ls='-', c='darkorange'),
         Line2D([0], [0], marker='', ls=':', c='black'),
         Line2D([0], [0], marker='s', ls='-', c='forestgreen')]

plt.legend(lines, ["SignatuRE", "GRU-ResNet", "Bespoke ResNet", "K2-RE"], loc='lower left', fontsize=12, ncol=2)

### Plot posteriors

In [ ]:
ma2df = pd.DataFrame()
for seed in range(20):
    sig = np.loadtxt("./MA2/{0}/signature_500_1_100_samples.txt".format(seed))
    gru = np.loadtxt("./MA2/{0}/gru-resnet_500_samples.txt".format(seed))
    hc = np.loadtxt("./MA2/{0}/hc_500_samples.txt".format(seed))
    k2 = np.loadtxt("./MA2/{0}/k2_500_samples.txt".format(seed))
    sig_df = pd.DataFrame(sig)
    sig_df["Method"] = "SignatuRE"
    gru_df = pd.DataFrame(gru)
    gru_df["Method"] = "GRU-ResNet"
    hc_df = pd.DataFrame(hc)
    hc_df["Method"] = "HC-ResNet"
    k2_df = pd.DataFrame(k2)
    k2_df["Method"] = "K2-RE"
    ma2df = ma2df.append(sig_df)
    ma2df = ma2df.append(gru_df)
    ma2df = ma2df.append(hc_df)
    ma2df = ma2df.append(k2_df)

ma2df.columns = [r"$\theta_1$", r"$\theta_2$", "Method"]

ma2_true_df = pd.DataFrame(ma2_true)
ma2_true_df["Method"] = "MH"
ma2_true_df.columns = [r"$\theta_1$", r"$\theta_2$", "Method"]
ma2df = ma2df.append(ma2_true_df)
print(ma2df)

sns.displot(ma2df, x=r"$\theta_1$", y=r"$\theta_2$", hue="Method", kind="kde")

In [ ]:
sns.displot(ma2df.loc[ma2df["Method"] != "K2-RE"], x=r"$\theta_1$", y=r"$\theta_2$", hue="Method", kind="kde")

# GSE

In [ ]:
smcabc = np.loadtxt("./gse_smcabc_0_samples.txt")

In [ ]:
plt.scatter(smcabc[:,0], smcabc[:,1], alpha=0.2)

In [ ]:
seeds = list(range(19, 29))
budgets = [50,100,200,500,1000]

## Compute metrics against SMC-ABC posterior

In [ ]:
gs_swd1, _, gs_md1 = compute_metrics.metrics(budgets, seeds,
                                  "./GSE/{0}/signature_{1}_1_50_samples.txt",
                                  smcabc, 
                                  sliced=False)

In [ ]:
gs_swd5, _, gs_md5 = compute_metrics.metrics(budgets, seeds,
                                  "./GSE/{0}/signature_{1}_5_50_samples.txt",
                                  smcabc, 
                                  sliced=False)

In [ ]:
gg_swd, _, gg_md = compute_metrics.metrics(budgets, seeds,
                                    "./GSE/{0}/gru-resnet_{1}_samples.txt",
                                  smcabc, 
                                  sliced=False)

In [ ]:
gk_swd5, _, gk_md5 = compute_metrics.metrics(budgets, seeds,
                                    "./GSE/{0}/k2_{1}_5_50_samples.txt",
                                  smcabc, 
                                  sliced=False)

In [ ]:
gk_swd1, _, gk_md1 = compute_metrics.metrics(budgets, seeds, 
                                    "./GSE/{0}/k2_{1}_1_50_samples.txt",
                                  smcabc, 
                                  sliced=False)

In [ ]:
gh_swd, _, gh_md = compute_metrics.metrics(budgets, seeds, 
                                  "./GSE/{0}/hc_{1}_samples.txt",
                                  smcabc, 
                                  sliced=False)

In [ ]:
dicts = [gg_swd, gk_swd1, gk_swd5, gs_swd1, gs_swd5, gh_swd]

for d in dicts:
    print(" & ".join(["{:.5f}".format(np.median(d[i])) for i in budgets]))
    print()